# 03 Function Calling | 02 Tools Extensions


## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create ChatClient

The `IChatClient` from the Microsoft.Extensions.AI NuGet package offers an abstraction which allows using models from multiple providers.

The `IChatClient` instance is created using the `ChatClientBuilder`. `UseFunctionInvocation()` from the ` Microsoft.Extensions.AI` nuget package ensures that tools (funcion calling) is enabled and supported.

```
IChatClient chatClient =
    new ChatClientBuilder(openAIChatClient.AsChatClient())
        .UseFunctionInvocation()
        .Build();
```




In [15]:
#r "nuget: Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3"
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.2"
#r "nuget: DotNetEnv, 3.1.1"

using Azure;
using Microsoft.Extensions.AI;
using Azure.AI.OpenAI;
using OpenAI;
using OpenAI.Chat;
using DotNetEnv;

//configuration file is created during environment creation
static string configurationFile = @"../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";    
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);

OpenAIClient openAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);
ChatClient openAIChatClient = openAIClient.GetChatClient(chatCompletionDeploymentName); 

IChatClient chatClient =
    new ChatClientBuilder(openAIChatClient.AsChatClient())
        .UseFunctionInvocation()
        .Build();

Console.WriteLine($"IChatClient created...");


Installed Packages Azure.AI.OpenAI, 2.2.0-beta.2 DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.Abstractions, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.AzureAIInference, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3

IChatClient created...


# Step 2: Define Tools

Two native functions: 

- `GetSportEventWinner()`
- `GetSportEventResult()`
  
are defined. The native functions are provided to the model as source for additional grounding information.

In [16]:
private string GetSportEventWinner(string sportEvent, string year)
{
    // Implement the logic to get the winner of the sport event.
    return "Kansas City Chiefs";
}

private string GetSportEventResult(string sportEvent, string year)
{
    // Implement the logic to get the result of the sport event.
    return "24:1";
}


Console.WriteLine($"Functions 'GetSportEventResult' and 'GetSportEventResult' defined...");

Functions 'GetSportEventResult' and 'GetSportEventResult' defined...


## Step 3: Define ChatOptions

The `AIFunctionFactory` creates the necessary description for the model to decide if function calling supports completing the provided prompt. 

In [17]:
ChatOptions chatOptions = new() { 
    Tools = [
        AIFunctionFactory.Create(GetSportEventWinner),
        AIFunctionFactory.Create(GetSportEventResult)
    ] 
};

Console.WriteLine("ChatOptions created...");

ChatOptions created...


## Step 4: Call Chat Completion

The chat completion endpoint of the deployed LLM is called with the previously defined `ChatOptions` which include the function or tool description.

The response from the LLM is checked ***automatically*** and local functions are called if requested by the LLM.

If no function or tool call is requested or after requested function or tool calls have been executed and the results have been provided to the model the result will be provided.

In [18]:
string prompt = "Who won the Super Bowl in 2024? Provide winning team and result.";

ChatResponse chatResponse = await chatClient.GetResponseAsync(prompt, chatOptions);

Console.WriteLine($"Model response: {chatResponse.Text}");

Model response: The Kansas City Chiefs won the Super Bowl in 2024, with a score of 24:1.
